In [1]:
#Dependencies
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
import re

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [139]:
#read training csv file from dataset
data_path = os.path.join('..', '..', 'jigsaw-unintended-bias-in-toxicity-classification', 'train.csv')
data_raw = pd.read_csv(data_path).dropna()

In [161]:
data_raw['asian'].unique()

array([0.00000000e+00, 1.00000000e+00, 3.00000000e-01, 5.00000000e-01,
       1.66666667e-01, 1.00000000e-01, 2.00000000e-01, 4.00000000e-01,
       3.80021716e-03, 8.00000000e-01, 7.00000000e-01, 1.10497238e-02,
       8.33333333e-01, 9.00000000e-01, 2.50000000e-01, 5.52181115e-04,
       7.50000000e-01, 6.00000000e-01, 1.08342362e-03, 6.32111252e-04,
       5.63697858e-04, 1.11111111e-01, 5.44069641e-04, 2.22222222e-01,
       5.45256270e-04, 9.68000000e-01, 3.33333333e-01, 5.46746856e-04,
       6.66666667e-01, 8.88888889e-01, 9.87694301e-01, 1.25000000e-01,
       1.10680686e-03, 3.86847195e-03, 6.42260758e-04, 2.38095238e-02,
       1.11856823e-03, 3.21336761e-03, 3.77155172e-03, 9.88188976e-01,
       5.45454545e-01, 6.47249191e-04, 7.16145833e-03, 1.81818182e-01,
       8.17661488e-04, 6.46412411e-04, 8.57142857e-01, 1.10375276e-03,
       1.42857143e-01, 9.41023979e-01, 6.45161290e-04, 5.57724484e-04,
       9.34227331e-01, 6.43915003e-04, 5.56792873e-04, 5.55555556e-01,
      

In [164]:
# create list of categories to reference
categories = list(data_raw.columns.values)
categories = categories[3:32]

# select 2000 random entries from dataset
data = data_raw
data = data_raw.loc[np.random.choice(data_raw.index, size=2000)]



In [165]:
def cleanHtml(sentence): #regex to clean punctuation
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext


def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned


def keepAlpha(sentence): #function to remove numbers
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

# stem data to combine words with similar meanings
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

#update list of stopwords from nltk
stop_words = set(stopwords.words('english'))
stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
#function to remove stop words
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

#function to make labels 0 if under 0.5, 1 if over 0.5
def binaryLabel(label):
    if label >= 0.5:
        return 1
    return 0

In [163]:
#apply cleaning functions to comment text
data['comment_text'] = data['comment_text'].str.lower()
data['comment_text'] = data['comment_text'].apply(cleanHtml)
data['comment_text'] = data['comment_text'].apply(cleanPunc)
data['comment_text'] = data['comment_text'].apply(keepAlpha)
data['comment_text'] = data['comment_text'].apply(stemming)
data['comment_text'] = data['comment_text'].apply(removeStopWords)
print(data.shape)

#convert ratings to binary labels
for category in categories:
    data[category] = data[category].map(binaryLabel)

(2000, 45)


,severe_toxicity,obscene,identity_attack,insult
536221,0.000000,0.000000,0.000000,0.000000
346710,0.000000,0.000000,0.000000,0.000000
1438911,0.000000,0.000000,0.000000,0.000000
276488,0.041667,0.055556,0.208333,0.722222
1442163,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...
707853,0.000000,0.000000,0.100000,0.100000
21620,0.000000,0.000000,0.000000,0.166667
352551,0.000000,0.000000,0.000000,0.000000
318403,0.012500,0.037500,0.037500,0.162500


In [177]:
#split data
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, random_state=42, test_size=0.30, shuffle=True)

In [178]:
train_text = train['comment_text']
test_text = test['comment_text']

In [179]:
# define vectorizer to turn comment strings into vectors 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(train_text)
vectorizer.fit(test_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents='unicode',
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [180]:
x_train = vectorizer.transform(train_text)
y_train = train[categories]

x_test = vectorizer.transform(test_text)
y_test = test[categories]
print(y_train)

severe_toxicity  obscene  identity_attack  insult  threat  asian  \
529162                 0        0                0       0       0      0   
1422440                0        0                0       0       0      0   
1629957                0        0                0       0       0      0   
147283                 0        0                0       0       0      0   
161660                 0        0                0       0       0      0   
...                  ...      ...              ...     ...     ...    ...   
1022306                0        0                0       0       0      0   
1528159                0        0                0       0       0      0   
672784                 0        0                0       1       0      0   
473117                 0        0                0       0       0      0   
1161090                0        0                0       0       0      0   

         atheist  bisexual  black  buddhist  ...  muslim  other_disability  \
529162

In [181]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputRegressor
from skmultilearn.problem_transform import LabelPowerset
from sklearn.tree import DecisionTreeClassifier

In [182]:

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])



for category in categories:
    printmd('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, train[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    print("\n")

**Processing severe_toxicity comments...**

Test accuracy is 1.0




**Processing obscene comments...**

Test accuracy is 0.9966666666666667




**Processing identity_attack comments...**

...

Test accuracy is 0.9883333333333333




**Processing transgender comments...**

Test accuracy is 0.9883333333333333




**Processing white comments...**

Test accuracy is 0.9516666666666667


